In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Training

In [2]:
# Load the dataset - January 2023
df = pd.read_parquet("data/yellow_tripdata_2023-01.parquet")
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [3]:
len(df.columns)

19

In [4]:
df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
df["duration"] = df["duration"].apply(lambda x: x.total_seconds() / 60)
df["duration"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [5]:
min(df["duration"])

-29.2

In [6]:
max(df["duration"])

10029.183333333332

In [7]:
np.mean(df["duration"])

np.float64(15.668995167330452)

In [8]:
np.std(df["duration"])

np.float64(42.59434429744777)

In [9]:
original_len = len(df)
original_len

3066766

In [10]:
df = df.query("duration >= 1 & duration <= 60")
df = df.reset_index()

In [11]:
updated_len = len(df)
updated_len

3009173

In [12]:
(updated_len / original_len) * 100

98.1220282212598

In [13]:
df

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,...,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,...,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,...,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,...,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,...,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3009168,3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,...,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3009169,3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,...,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3009170,3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,...,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3009171,3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,...,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [14]:
df = df.reindex(columns=["PULocationID", "DOLocationID", "duration"])

In [15]:
df

,PULocationID,DOLocationID,duration
0,161,141,8.433333
1,43,237,6.316667
2,48,238,12.750000
3,138,7,9.616667
4,107,79,10.833333
...,...,...,...
3009168,107,48,13.983333
3009169,112,75,19.450000
3009170,114,239,24.516667
3009171,230,79,13.000000


In [16]:
df["PULocationID"] = df["PULocationID"].astype(str)
df["DOLocationID"] = df["DOLocationID"].astype(str)
df.dtypes

PULocationID     object
DOLocationID     object
duration        float64
dtype: object

In [17]:
X = df.drop(columns="duration")
y = df["duration"]

In [18]:
X = X.to_dict(orient="records")

In [19]:
dv = DictVectorizer()
X_train = dv.fit_transform(X)
X_train.shape

(3009173, 515)

In [20]:
model = LinearRegression()
model.fit(X_train, y)

LinearRegression()

In [21]:
y_preds = model.predict(X_train)
rmse = root_mean_squared_error(y, y_preds)
rmse

7.649261936284003

# Validation

In [ ]:
# Load the dataset - February 2023
df = pd.read_parquet("data/yellow_tripdata_2023-02.parquet")

In [ ]:
# Same preprocessing steps
df["duration"] = (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"])
df["duration"] = df["duration"].apply(lambda x: x.total_seconds() / 60)

In [24]:
df = df.query("duration >= 1 & duration <= 60")
df = df.reset_index()

In [25]:
df = df.reindex(columns=["PULocationID", "DOLocationID", "duration"])

In [26]:
df["PULocationID"] = df["PULocationID"].astype(str)
df["DOLocationID"] = df["DOLocationID"].astype(str)

In [27]:
X = df.drop(columns="duration")
y = df["duration"]

In [28]:
X = X.to_dict(orient="records")

In [29]:
X_val= dv.transform(X)

In [30]:
y_preds = model.predict(X_val)

In [31]:
rmse = root_mean_squared_error(y, y_preds)
rmse

7.811818654341152